In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [5]:
path = '/content/drive/MyDrive/filtered_data/processed_data/'
def read_data(label2id):
    X = []
    Y = []
    for label in os.listdir(path):
        for img_path in os.listdir(os.path.join(path, label)):
            img = cv2.imread(os.path.join(path, label, img_path))
            X.append(img)
            Y.append(label2id[label])
    return X, Y

In [6]:
label2id = {'Amsterdam-Landscape': 0,
            'Dubai-landscape': 1,
            'Hanoi-Landscape': 2,
            'Kyoto-Landscape': 3,
            'London-landscape': 4,
            'Maldives-landscape': 5,
            'Moscow-Landscape': 6,
            'Newyork-landscape': 7,
            'Paris-Landscape': 8,
            'Rabat-landscape': 9,
            'Rio-De-Janeiro-Landscape': 10,
            'Roma-Landscape': 11,
            'Singapore-landscape': 12,
            'Sydney-landscape': 13,
            'Venezia-landscape': 14}
# X, Y = read_data(label2id)
# print(len(X))
# print(len(Y))

Bởi vì data quá nhiều nên sẽ thử xử lý hướng trích xuất sift_features cho từng ảnh và chỉ lưu descriptor của từng ảnh

In [7]:
def extract_sift_features(img):
    # image_descriptors = []
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(img, None)
    # image_descriptors.append(cv2.KeyPoint_convert(kp))
    return des
    # return image_descriptors

In [ ]:
test_img = cv2.imread('/content/drive/MyDrive/filtered_data/processed_data/Amsterdam-Landscape/0.jpg')
img_descriptor = extract_sift_features(test_img)

In [ ]:
img_descriptor.shape

(5413, 128)

In [ ]:
# winSize = (64,64)
# blockSize = (16,16)
# blockStride = (8,8)
# cellSize = (8,8)
# nbins = 9
# derivAperture = 1
# winSigma = 4.
# histogramNormType = 0
# L2HysThreshold = 2.0000000000000001e-01
# gammaCorrection = 0
# nlevels = 64
# hog_descriptor = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
#                         histogramNormType,L2HysThreshold,gammaCorrection,nlevels)

In [8]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)

In [15]:
from tqdm.notebook import tqdm
image_descriptors = []
all_descriptors = []
Y = []
# hog_descriptor = cv2.HOGDescriptor()
for label in os.listdir(path):
  if 'csv' not in os.listdir(os.path.join(path, label)):
    for img_path in tqdm(os.listdir(os.path.join(path, label))):
        img = cv2.imread(os.path.join(path, label, img_path))
        try:
          img = cv2.resize(img, (256, 256))
          # img_descriptor = hog_descriptor.compute(img)
          # reduced_img_descriptor = svd.fit_transform(img_descriptor.reshape(-1, 1))
          img_descriptor = extract_sift_features(img)
          if img_descriptor is not None:
            image_descriptors.append([des for des in img_descriptor])
            all_descriptors.extend(img_descriptor)
            Y.append(label2id[label])
          else:
            print(label + img_path)
        except:
          print(label + img_path)

  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/908 [00:00<?, ?it/s]

Kyoto-Landscape776.jpg


  0%|          | 0/868 [00:00<?, ?it/s]

  0%|          | 0/620 [00:00<?, ?it/s]

...

  0%|          | 0/480 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/331 [00:00<?, ?it/s]

Rio-De-Janeiro-Landscape826.jpg


  0%|          | 0/526 [00:00<?, ?it/s]

In [16]:
print(len(image_descriptors))
print(len(Y))

9572
9572


In [21]:
svd_image_descriptors = []
for img_descriptor in image_descriptors:
  svd_image_descriptors.append(svd.fit_transform(img_descriptor))

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_truncated_svd.py:268: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_truncated_svd.py:268: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


In [25]:
print(len(svd_image_descriptors))
print(svd_image_descriptors[0].shape)

9572
(466, 5)


In [19]:
for i in range(10):
    print('Image {} has {} descriptors'.format(i, len(image_descriptors[i])))

Image 0 has 466 descriptors
Image 1 has 677 descriptors
Image 2 has 449 descriptors
Image 3 has 354 descriptors
Image 4 has 139 descriptors
Image 5 has 579 descriptors
Image 6 has 537 descriptors
Image 7 has 410 descriptors
Image 8 has 411 descriptors
Image 9 has 211 descriptors


In [ ]:
all_descriptors = np.array(all_descriptors)

<ipython-input-25-88853e59d96e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_descriptors = np.array(all_descriptors)


In [ ]:
all_descriptors[0]


In [10]:
all_descriptors = svd.fit_transform(all_descriptors)

In [11]:
all_descriptors.shape

(5326841, 5)

In [12]:
import time
def kmeans_bow(all_descriptors, num_clusters):
    start = time.time()
    bow_dict = []
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(all_descriptors)
    bow_dict = kmeans.cluster_centers_
    print('Process time: ', time.time() - start)

    return bow_dict

In [13]:
import pickle
num_clusters = 50
BoW = kmeans_bow(all_descriptors, num_clusters)
pickle.dump(BoW, open('bow_dictionary.pkl', 'wb'))
print(len(BoW))


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Process time:  1417.1434473991394
50


In [20]:
BoW.shape

(50, 5)

In [32]:
def create_features_bow(image_descriptors, BoW, num_clusters):
    X_features = []
    removed_idx = []
    for i in range(len(image_descriptors)):
        try:
          features = np.array([0] * num_clusters)
          if image_descriptors[i] is not None:
              distance = cdist(image_descriptors[i], BoW)
              idx =  np.argmin(distance, axis=1)
              print(distance)
              for j in idx:
                  features[j] += 1
          X_features.append(features)
        except:
          removed_idx.append(i)
          continue
    return X_features, removed_idx

In [33]:
X_features, removed_idx = create_features_bow(svd_image_descriptors, BoW, num_clusters)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  325.44275845]
 ...
 [318.17551377 240.68558293 241.61067933 ... 306.0811864  304.49452065
  252.90383477]
 [270.70157185 264.18303681 278.35386151 ... 310.46606497 354.83999869
  223.9524101 ]
 [165.19171233 238.01237146 249.80215621 ... 323.25911841 380.35372521
  274.77017508]]
[[389.41936262 257.97432941 377.06677649 ... 147.6210084  238.23780907
  297.04565199]
 [222.88091103 224.41345529 232.37894752 ... 297.14631481 339.32312561
  247.60520372]
 [265.15199007 174.03187101 128.70690988 ... 307.85945413 283.08328229
  276.09032371]
 ...
 [395.92962564 248.02213846 369.85402741 ... 164.65686699 233.51423864
  307.00610534]
 [337.17638709 227.80880133 336.11691238 ... 190.65396654 259.20382692
  296.29172471]
 [379.77386175 257.37492475 359.64797304 ... 221.19050571 273.39022861
  293.39523756]]
[[398.73950346 182.3627441  124.84512096 ... 342.56969777 209.81790806
  371.57476619]
 [318.82422623 253.91625439 379.98029907 ... 

In [34]:
print(len(X_features))

9568


In [35]:
print(removed_idx)

[1689, 1853, 2051, 2333]


In [36]:
for idx in removed_idx:
  del Y[idx]
print(len(Y))

9568


In [38]:
from sklearn.model_selection import train_test_split

X_train = []
X_test = []
Y_train = []
Y_test = []

X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y, test_size=0.3)

print(len(X_train))
print(len(X_test))

6697
2871


In [39]:
from sklearn.svm import SVC
classifier = SVC(C = 30)
classifier.fit(X_train, Y_train)

SVC(C=30)

In [40]:
classifier.score(X_train, Y_train)


0.939077198745707

In [41]:
classifier.score(X_test, Y_test)

0.39359108324625564

In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
dict_param = {"C": [0.1, 10, 30],
              "kernel": ['linear', 'poly', 'rfb']}
svm = SVC()

clf = GridSearchCV(estimator=svm, param_grid=dict_param, cv=5)
clf.fit(X_train, Y_train)
best_svm = clf.best_estimator_

print("Train accuracy:", best_svm.score(X_train, Y_train))
print("Test accuracy:", best_svm.score(X_test, Y_test))